In [5]:
import os
import torch
import pandas as pd
from pathlib import Path
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple, Dict

In [19]:
# create a custom dataset class


class LungDatasets(Dataset):
    def __init__(self, sources, root_dir:Path,
                 partitions: List[str] = ['train', 'validation'],
                 transform=None):
        self.sources = sources
        self.root_dir = root_dir
        self.partitions = partitions
        self.transform = transform

    def __len__(self):
        total_images = 0
        for source in self.sources:
            # always 2 image in scans (inhale and exhale)
            total_images += len(list((self.root_dir/f'{source}/scans').iterdir()))//2
        return total_images
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = read_image(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

In [20]:
l2r = LungDatasets(sources=['learn2reg'], root_dir=Path('data'))

In [21]:
len(l2r)

20